# Processing heat fluxes 
(crop to gom, slice to glorys time period, save to netcdf)

In [1]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt

In [2]:
sw_ds = xr.open_mfdataset('/work5/stb/ERA5/Solar-radiation/ERA5_24hr_surface_net_solar_radiation_*.nc', chunks={})
lw_ds = xr.open_mfdataset('/work5/stb/ERA5/Longwave-radiation/ERA5_24hr_surface_net_thermal_radiation_*.nc', chunks={})
sh_ds = xr.open_mfdataset('/work5/stb/ERA5/Sensible-heat/ERA5_24hr_surface_sensible_heat_flux_*.nc', chunks={})
lh_ds = xr.open_mfdataset('/work5/stb/ERA5/Latent-heat/ERA5_24hr_surface_latent_heat_flux_*.nc', chunks={})

In [14]:
lh_ds

<xarray.Dataset> Size: 137GB
Dimensions:    (time: 16436, latitude: 721, longitude: 1440)
Coordinates:
  * longitude  (longitude) float32 6kB 0.0 0.25 0.5 0.75 ... 359.2 359.5 359.8
  * latitude   (latitude) float32 3kB 90.0 89.75 89.5 ... -89.5 -89.75 -90.0
  * time       (time) datetime64[ns] 131kB 1979-01-01 1979-01-02 ... 2023-12-31
Data variables:
    slhf       (time, latitude, longitude) float64 137GB dask.array<chunksize=(365, 721, 1440), meta=np.ndarray>
Attributes:
    Conventions:  CF-1.6
    history:      2025-02-26 18:32:16 GMT by grib_to_netcdf-2.39.0: grib_to_n...

In [15]:
time_slice = dict(time=slice('1993', '2021-06-30'))
gom = dict(longitude=slice(262, 279), latitude=slice(31, 17.5))

sw = sw_ds.ssr.sel(**gom).sel(**time_slice)
lw = lw_ds.str.sel(**gom).sel(**time_slice)
sh = sh_ds.sshf.sel(**gom).sel(**time_slice)
lh = lh_ds.slhf.sel(**gom).sel(**time_slice)

In [18]:
heat_fluxes = xr.merge([sw, lw, sh, lh])

In [19]:
heat_fluxes

<xarray.Dataset> Size: 1GB
Dimensions:    (longitude: 69, latitude: 55, time: 10408)
Coordinates:
  * longitude  (longitude) float32 276B 262.0 262.2 262.5 ... 278.5 278.8 279.0
  * latitude   (latitude) float32 220B 31.0 30.75 30.5 30.25 ... 18.0 17.75 17.5
  * time       (time) datetime64[ns] 83kB 1993-01-01 1993-01-02 ... 2021-06-30
Data variables:
    ssr        (time, latitude, longitude) float64 316MB dask.array<chunksize=(365, 55, 69), meta=np.ndarray>
    str        (time, latitude, longitude) float64 316MB dask.array<chunksize=(365, 55, 69), meta=np.ndarray>
    sshf       (time, latitude, longitude) float64 316MB dask.array<chunksize=(365, 55, 69), meta=np.ndarray>
    slhf       (time, latitude, longitude) float64 316MB dask.array<chunksize=(365, 55, 69), meta=np.ndarray>
Attributes:
    units:          J m**-2
    long_name:      Surface net short-wave (solar) radiation
    standard_name:  surface_net_downward_shortwave_flux

In [21]:
pwd

'/work5/stb/MHW-gfdl'

In [ ]:
heat_fluxes.to_netcdf('/work5/stb/ERA5/era-heat-fluxes.nc')